In [2]:
import numpy as np
import torch
from torchvision import transforms, datasets

In [11]:
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [7]:
def Select_Class(trainset, testset, batch=64):
    val_ratio = 0.1
    train_size = len(trainset)

    split = int(val_ratio * train_size)

    train_dataset, val_dataset = random_split(trainset, [train_size-split, split])

    custom_train_dataloader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    custom_val_dataloader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
    custom_test_dataloader = DataLoader(testset, batch_size=batch, shuffle=True)

    return custom_train_dataloader, custom_val_dataloader, custom_test_dataloader

In [8]:
def Normalize_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
    testset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
    return trainset, testset

normalized_trainset, normalized_testset = Normalize_dataset()
normalized_train_dataloader, normalized_val_dataloader, normalized_test_dataloader = Select_Class(normalized_trainset, normalized_testset)

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [40]:
class SVC(nn.Module):
    def __init__(self):
        super(SVC, self).__init__()
        self.linear = nn.Linear(3*32*32, 10, device=device)  # 10개 클래스 출력을 위한 차원 변경

    def forward(self, X):
        return self.linear(X)

    def fit(self, train_dataloader, optimethod="SGD", learning_rate=0.01, epochs=10):
        optimizer = optim.SGD(self.parameters(), lr=learning_rate) if optimethod == "SGD" else optim.Adam(self.parameters(), lr=learning_rate)

        # 다중 클래스 분류를 위한 손실 함수 (예: 크로스 엔트로피)
        criterion = nn.CrossEntropyLoss()

        self.train()
        for epoch in range(epochs):
            total_loss = 0
            for X, y in train_dataloader:
                X.to(device)
                y.to(device)
                X = torch.flatten(X, 1, -1)
                y = y.long()  # 클래스 레이블을 정수형으로 변환
                optimizer.zero_grad()
                outputs = self(X)
                loss = criterion(outputs, y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_dataloader):.4f}')
        print(f'* Final loss: {total_loss:.3f}\n* Train accuracy: {self.predict(train_dataloader):.2f}%')

    def predict(self, dataloader):
        self.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for X, y in dataloader:
                X.to(device)
                y.to(device)
                X = torch.flatten(X, 1, -1)
                outputs = self(X)
                _, predicted = torch.max(outputs.data, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
        accuracy = 100 * correct / total
        return accuracy

In [41]:
svc = SVC().to(device)

svc.fit(normalized_train_dataloader, learning_rate=0.001, optimethod="Adam", epochs=1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)